In [179]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from time import sleep
import os #os.mkdir('abc') / os.rmdir('abc') / os.path.isdir('abc')

In [ ]:
#함수 정의
def make_row(tr):
    try:
        rank = tr.find('strong').text
        title = tr.find('p',{'class':'title'}).find('a').text
    except:
        title = tr.find('p',{'class':'title'}).find_all('span')[1].text
        artist = tr.find('p',{'class':'artist'}).find('a').text
        try:
            album = tr.find('a',{'class':'album'}).text
        except:
            album = tr.find('span',{'class':'album'}).text
            img_url = tr.find('img').get('src').split('?')[0]
            row = [rank,title,artist,album]
    else:
        artist = tr.find('p',{'class':'artist'}).find('a').text
        album = tr.find('a',{'class':'album'}).text
        img_url = tr.find('img').get('src').split('?')[0]
        row = [rank,title,artist,album,img_url]
        
    return {'info':row,'url':img_url}


In [175]:

url = 'https://music.bugs.co.kr/chart'
#response는 headers와 content(데이터, 텍스트)로 구성 되어있다.
response = requests.get(url)
#response.headers 안에 headers가 dict로 담겨있다.
#response.content 안에 content가 DATA(Binary)로 담겨있다. image, zip, hwp와 같은 형식
#response.text 안에 contetn가 text로 담겨있다.

#response.headers['Content-Type'] 안에 데이터의 종류가 담겨있다.

text = response.text
text[:259]

html = bs(text, 'html.parser')
html

tr_list = html.find('table',{'class':'list trackList byChart'}).find('tbody').find_all('tr')
tr_list


rows = []
for tr in tr_list:
    row = make_row(tr)
    rows.append(row)
    
df = pd.DataFrame(rows, columns = ['rank','title','artist','album']) 
df.set_index('rank', inplace = True)
df.to_csv('csv/bugs1.csv',mode="w",encoding='euc-kr')
df

,title,artist,album
rank,,,
1,밤양갱,비비(BIBI),밤양갱
2,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’
3,나는 아픈 건 딱 질색이니까,(여자)아이들,2
4,EASY,LE SSERAFIM (르세라핌),EASY
5,홀씨,아이유(IU),The Winning
...,...,...,...
96,달빛에 그려지는,미연 ((여자)아이들),연인 OST Part.4
97,Chill Kill,Red Velvet (레드벨벳),Chill Kill - The 3rd Album
98,운이 좋았지,권진아,나의 모양


In [100]:
tr = tr_list[0] #tr_list가 find_all함수로 끝났기 때문에 인덱스[0]을 넣어야한다.
title = tr.find('p',{'class':'title'}).find('a')
rank = tr.find('strong').text
artist = tr.find('p',{'class':'artist'}).find('a').text
#album = tr.find('p',{'class':'left'}).find('a').text

album = tr.find('a',{'class':'album'}).text
album

'밤양갱'

In [ ]:
year = 2018
month = 1
#day = 1

for day in range(11,13):
    date = f'{year}{month:0>2}{day:0>2}'
    path = f'bugs_{date}'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'
   
    response = requests.get(url)
    text = response.text
    html = bs(text, 'html.parser')
    tr_list = html.find('table',{'class':'list trackList byChart'}).find('tbody').find_all('tr')

    rows = []
    for tr in tr_list:
        result = bugs_row(tr) #에러
        row = result['info']
        img_url = result['url']
        
        response2 = requests.get(img_url)
        content = response2.content
        filename = f'{path}/image_{date}_{row[0]}.jpeg'
        file = open(filename,'wb')
        file.write(content)
        file.close()

        rows.append(row)
        
    df = pd.DataFrame(rows, columns = ['rank','title','artist','album']) 
    df.set_index('rank', inplace = True)
    filename = f'{path}/bugs_chart_{date}.csv'
    df.to_csv(filename, encoding='utf-8-sig') #왜 오류?
    sleep(5)   #반복 처리시 sleep처리를 안하면 디도스로 취급해버린다!!중요

## 날짜 비어있는 칸에 0넣기

In [133]:
year = 2024
month = 1
day = 1



"""if month < 10:
    month = f'0{month}'
if day < 10:
    day = f'0{day}'
"""
date = f'{year}{month}{day}'
date = f'{year}{month:0>2}{day:0>2}' #f열 포매팅 기법 속성값 ':0>2'는 빈공간이 나올경우 2칸까지 공간 만들고 빈칸을 0으로 채운다.
date

'20240101'

In [137]:
date = '20180111'
url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'
response = requests.get(url)
text = response.text
#text[:259]
html = bs(text, 'html.parser')

tr_list = html.find('table',{'class':'list trackList byChart'}).find('tbody').find_all('tr')
rank37 = tr_list[36]
rank38 = tr_list[37]


print(rank37.find('p',{'class':'title'}))
print(rank38.find('p',{'class':'title'}).find_all('span')[1])
rank38.find('p',{'class':'title'}).find_all('span','') #속성이 없는 'span'태그를 반환해준다.

<p adult_yn="N" class="title">
<a adultcheckval="1" aria-label="새창" href="javascript:;" onclick="bugs.wiselog.area('list_tr_09_chart');bugs.music.listen('4898930',true);
" title="There's Nothing Holdin' Me Back">There's Nothing Holdin' Me Back</a>
</p>
<span>My My My!</span>


'rows = []\nfor tr in tr_list:\n    row = make_row(tr)\n    rows.append(row)\n\ndf = pd.DataFrame(rows, columns = [\'rank\',\'title\',\'artist\',\'album\']) \ndf.set_index(\'rank\', inplace = True)\ndf.to_csv(f\'csv/bugs{date}.csv\',mode="w",encoding=\'euc-kr\')\nsleep(3)   #반복 처리시 sleep처리를 안하면 디도스로 취급해버린다!!중요'

In [173]:
rank37 = tr_list[36]

url = rank37.find('img').get('src').split('?')[0] #이미지 소스의 url주소 뽑기
response = requests.get(url)
content = response.content

#이미지 저장
file = open('csv/그림.jpg','wb')
file.write(content)
file.close()

"\ni = 0\ni = i+1\nimg = dload.save(src, f'img{i}.jpg')"

In [ ]:
def bugs_row(tr):
    rank = tr.find('strong').text
    try:
        title = tr.find('p', {'class':'title'}).find('a').text
    except:
        title = tr.find('p', {'class':'title'}).find('span', '').text
    artist = tr.find('p', {'class':'artist'}).find('a').text
    try:
        album = tr.find('a', {'class':'album'}).text
    except:
        album = tr.find('span', {'class':'album'}).text
    img_url = tr.find('img').get('src').split('?')[0]
    row  = [rank, title, artist, album]
    return {'info':row, 'url':img_url}